In [1]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

In [2]:
from pydantic import BaseModel
from typing import List


class Words(BaseModel):
    word: str
    meaning_eng: str
    meaning_native: str

class WordList(BaseModel):
    words: List[Words]

In [ ]:
from crewai import Crew, Agent, Task


word_finder = Agent(
    role="Searching the meaning of words",
    goal="Find the meaning of given words in English and given native language.",
    backstory="""
        You are finding the meaning of English words in dictionaries for who studying English.
        Find the meaning of given words in English first.
        Explain the words briefly in the tone of dictionaries.
        Secondly, find what it means in native language(in other words, how the words can translate in native language).
        You don't need to explain the native word wordily.
        DO NOT use English when you translate the words in native language.
        The words will be given as a comma-separated list.
    """,
    verbose=True,
    allow_delegation=False,
)

In [10]:
searching_task = Task(
    description="Find the meaning of given words in English and {native_lang} by searching dictionaries: {word_list}",
    agent=word_finder,
    expected_output="A List of Words Object",
    output_json=WordList,
)

In [11]:
crew = Crew(
    tasks=[
        searching_task,
    ],
    agents=[
        word_finder,
    ],
    verbose=2,
)

2025-08-29 13:10:29,469 - 140454591799424 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [12]:
result = crew.kickoff(
    inputs=dict(
        native_lang="Korean",
        word_list="amygdala",
    )
)

 [DEBUG]: == Working Agent: Searching the meaning of words
 [INFO]: == Starting Task: Find the meaning of given words in English and Korean by searching dictionaries: amygdala


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer  
Final Answer: 

**Word**: Amygdala  
**Meaning in English**: The amygdala is a small, almond-shaped cluster of nuclei located deep within the temporal lobes of the brain. It plays a crucial role in processing emotions, particularly fear and pleasure. The amygdala is involved in the formation of emotional memories and is key to the fight-or-flight response.

**Meaning in Korean**: 편도체

> Finished chain.
 [DEBUG]: == [Searching the meaning of words] Task output: {
  "words": [
    {
      "word": "Amygdala",
      "meaning_eng": "The amygdala is a small, almond-shaped cluster of nuclei located deep within the temporal lobes of the brain. It plays a crucial role in processing emotions, particularly fear and pleasure. The amygdala is

In [7]:
import json

crewai_response = json.loads(result)
words = crewai_response['words']

words

[{'word': 'amygdala',
  'meaning_eng': 'The amygdala is an almond-shaped cluster of nuclei located deep within the temporal lobe of the brain. It is involved in the processing of emotions, fear response, and memory.',
  'meaning_native': '편도체'}]

In [8]:
from googlesheets_utils import GooglesheetUtils

googlesheet = GooglesheetUtils(spreadsheet_id='1hFNuCdmySJodQM5qsR5FJ6pkPLQc5DbXwP7h74pwTs8')

values = [[word['word'], word['meaning_eng'], word['meaning_native']] for word in words]
googlesheet.append_data('Behave!B4', values)



3 cells appended.


{'spreadsheetId': '1hFNuCdmySJodQM5qsR5FJ6pkPLQc5DbXwP7h74pwTs8',
 'tableRange': 'Behave!B4:E307',
 'updates': {'spreadsheetId': '1hFNuCdmySJodQM5qsR5FJ6pkPLQc5DbXwP7h74pwTs8',
  'updatedRange': 'Behave!B308:D308',
  'updatedRows': 1,
  'updatedColumns': 3,
  'updatedCells': 3}}